In [1]:
import warnings
import pandas as pd
pd.options.plotting.backend = 'plotly'
warnings.filterwarnings('ignore')

In [2]:
# Values in 1952, queried from the autumn database, hard-coded
death_rates = pd.Series(
    [0.048912, 0.005655, 0.003052, 0.009387, 0.0141716], 
    index=[0, 5, 15, 35, 50]
)

In [3]:
upper_age = 100
ages = range(upper_age)

In [4]:
# Get the death rates applicable to each age
life_table = pd.DataFrame(index=ages, columns=['death_rate'])
for a in ages:
    idx = next((i for i, age in enumerate(death_rates.index) if age > a), 0)  # The zero is a trick to get the last element when one is subtracted
    life_table.loc[a, 'death_rate'] = death_rates.iloc[idx - 1]

In [5]:
df = life_table.reset_index()
df = df.rename(columns={'index': 'age', 'death_rate': 'qx'}) 
# qx: is the probability that a person aged exactly x dies before exact age (x+1), i.e., the probability of dying in the next year. Hence: qx= dx / lx
# lx: is the number of persons alive aged x in a life table.
df['lx'] = 100000
# dx: is the number of persons who die aged x last birthday.
for i in range(1, len(df)):
    df.loc[i, 'lx'] = df.loc[i - 1, 'lx'] - (df.loc[i - 1, 'qx'] * df.loc[i - 1, 'lx'])
    df["dx"] = 0
for x in range(100):
    if x >=99:
        df["dx"][x] = df["lx"][x]
    else:
        df["dx"][x] = df["lx"][x]-df["lx"][x+1]
#μx: is the force of mortality, i.e., represents the instantaneous rate at which people are dying.
df["μx"] = 0
df["μx"][0] = 2.0692602739726 * df["dx"][0] /(2*df["lx"][0])
for x in range(1,100):
    df["μx"][x] = (df["dx"][x-1]+df["dx"][x])/(2*df["lx"][x])

#Tx: is the total years of life to be lived by those aged exactly x (not the random variable Tx) until they all die
df["Tx"] = 0.000
for x in range(100):
    df["Tx"][x] = 0
    for y in range(x,100):
        df["Tx"][x] += df["lx"][y]
# e0x: is the complete expectation of life, i.e., the expectation of life allowing for the whole of the lifetime of the individual (years and days).
df["e0x"] = df["Tx"]/df["lx"] - 0.5
# total life expectancy, Lx: is the number of years of life lived between ages x and (x+1) of those currently aged x.
df["Lx"] = 0.000
for x in range(100):
    df["Lx"][x] = df["Tx"][x]-df["lx"][x]
# ex: is the curtate expectation of life, i.e., the expectation of life where only complete years of life count.
df["ex"] = df["Lx"]/df["lx"]
df

,age,qx,lx,dx,μx,Tx,e0x,Lx,ex
0,0,0.048912,100000.000000,4891.200000,0.050606,5.774515e+06,57.245149,5.674515e+06,56.745149
1,1,0.048912,95108.800000,4651.961626,0.050170,5.674515e+06,59.163405,5.579406e+06,58.663405
2,2,0.048912,90456.838374,4424.424879,0.050170,5.579406e+06,61.180312,5.488949e+06,60.680312
3,3,0.048912,86032.413496,4208.017409,0.050170,5.488949e+06,63.300944,5.402917e+06,62.800944
4,4,0.048912,81824.396087,4002.194861,0.050170,5.402917e+06,65.530635,5.321092e+06,65.030635
...,...,...,...,...,...,...,...,...,...
95,95,0.014172,31589.631852,447.675627,0.014273,1.535344e+05,4.360278,1.219448e+05,3.860278
96,96,0.014172,31141.956225,441.331347,0.014273,1.219448e+05,3.415771,9.080281e+04,2.915771
97,97,0.014172,30700.624878,435.076976,0.014273,9.080281e+04,2.457686,6.010218e+04,1.957686
98,98,0.014172,30265.547903,428.911239,0.014273,6.010218e+04,1.485828,2.983664e+04,0.985828
